Credit to: 
* Brian Coffey's talk here: http://multithreaded.stitchfix.com/blog/2015/12/15/d3-jupyter/ with associated code here: https://github.com/stitchfix/d3-jupyter-tutorial in which it is revealed how to jam D3 into Jupyter notebooks.

* Mike Bostock's Force Directed Tree code, here: https://bl.ocks.org/mbostock/1138500

* Daniel Borowski's Fibonacci Heap implementation, here: https://github.com/danielborowski/fibonacci-heap-python which helped clarify my own implementation

In [1]:
from IPython.core.display import display, HTML
from string import Template
import json

In [2]:
HTML('<script src="lib/d3/d3.min.js"></script>')

In [3]:
html_template = Template('''
<style> $css_text </style>
<div id="graph-div"></div>
<script> $js_text </script>
''')

In [4]:
css_text = '''
circle {
  stroke-width: 1.5px;
}

line {
  stroke: #999;
  stroke-width: 1.5px;
}
'''

In [5]:
js_text_template = Template('''
var width = 960,
    height = 500,
    radius = 6;

var fill = d3.scale.category20();

var force = d3.layout.force()
    .charge(-120)
    .linkDistance(30)
    .size([width, height]);

var svg = d3.select("#graph-div").append("svg")
    .attr("width", width)
    .attr("height", height);

var json = $data;

  var link = svg.selectAll("line")
      .data(json.links)
    .enter().append("line");

  var node = svg.selectAll("circle")
      .data(json.nodes)
    .enter().append("circle")
      .attr("r", radius - .75)
      .style("fill", function(d) { return fill(d.group); })
      .style("stroke", function(d) { return d3.rgb(fill(d.group)).darker(); })
      .call(force.drag);
  
  node.append("title")
      .text(function(d) { return d.name; });

  force
      .nodes(json.nodes)
      .links(json.links)
      .on("tick", tick)
      .start();

  function tick(e) {
    var k = 6 * e.alpha;

    // Push sources up and targets down to form a weak tree.
    link
        .each(function(d) { d.source.y -= k, d.target.y += k; })
        .attr("x1", function(d) { return d.source.x; })
        .attr("y1", function(d) { return d.source.y; })
        .attr("x2", function(d) { return d.target.x; })
        .attr("y2", function(d) { return d.target.y; });

    node
        .attr("cx", function(d) { return d.x; })
        .attr("cy", function(d) { return d.y; });

  }
''')

In [6]:
from fibheapsifter import FibHeap

In [7]:
num_heap = FibHeap()
for num in range(0, 33):
    num_heap.insert(num)

In [53]:
def convert_heap_to_json_data(fibheap):
    def get_all_children(node):
        nodes = []
        if node.children:
            for child_node in node.children:
                child_nodes = get_all_children(child_node)
                nodes.extend(child_nodes)
            nodes.extend(node.children)
        return nodes
    
    def get_all_nodes(heap):
        nodes = []
        for node in heap.root_list:
            children = get_all_children(node)
            nodes.extend(children)
            nodes.extend([node])
        return nodes
    all_nodes = get_all_nodes(fibheap)
    
    for index, node in enumerate(all_nodes):
        node.index = index
    data = {"nodes": [{"name": str(node.item)} for node in all_nodes],
            "links": [{"source": node.parent.index, "target": node.index} for node in all_nodes if node.parent]}
    return json.dumps(data)

In [9]:
data = convert_heap_to_json_data(num_heap)

In [10]:
js_text = js_text_template.substitute({'data': data})

In [59]:
HTML(html_template.substitute({'css_text': css_text, 'js_text': js_text}))

In [12]:
num_heap.extract_minimum()

0

In [49]:
def crank_heap(heap):
    data = convert_heap_to_json_data(heap)
    js_text = js_text_template.substitute({'data': data})
    return HTML(html_template.substitute({'css_text': css_text, 'js_text': js_text}))

In [14]:
crank_heap()

In [50]:
import functools

#@functools.lru_cache(maxsize=None)
def prioritize_or_equal(item_a, item_b):
    print("a: {}".format(item_a))
    print("b: {}".format(item_b))
    choice = input("More important? a/b/(e)qual ")
    if choice == 'b':
        return 'greater'
    elif choice == 'a':
        return 'less'
    else:
        return 'equal'

@functools.total_ordering
class TODO:
    def __init__(self, text):
        self.text = text

    def __repr__(self):
        return "TODO({})".format(self.text)

    def __str__(self):
        return self.text

    def __eq__(self, other):
        if self.text < other.text:
            return prioritize_or_equal(self.text, other.text) == 'equal'
        else:
            return prioritize_or_equal(other.text, self.text) == 'equal'

    def __lt__(self, other):
        if self.text < other.text:
            return prioritize_or_equal(self.text, other.text) == 'greater'
        else:
            return prioritize_or_equal(other.text, self.text) == 'less'


class HeapsifterHeap(object):
    def __init__(self):
        self.heap = FibHeap()
        
    def insert_todo(self, text):
        todo = TODO(text)
        self.heap.insert(todo)
        return crank_heap(self.heap)
        
    
    def show_top(self):
        print(self.heap.minimum)
    
    def complete_top(self):
        self.heap.extract_minimum()
        print("New top priority: {}".format(self.heap.minimum))
        return crank_heap(self.heap)

In [51]:
my_heap = HeapsifterHeap()

In [54]:
my_heap.insert_todo("File taxes.")

a: File taxes.
b: File taxes.
More important? a/b/(e)qual a


In [55]:
my_heap.insert_todo("Mow lawn.")

a: File taxes.
b: Mow lawn.
More important? a/b/(e)qual a


In [56]:
my_heap.show_top()

TODO(File taxes.)


In [57]:
my_heap.insert_todo("Edit footage.")

a: Edit footage.
b: File taxes.
More important? a/b/(e)qual b


In [58]:
my_heap.complete_top()

a: File taxes.
b: Mow lawn.
More important? a/b/(e)qual a
a: Edit footage.
b: Mow lawn.
More important? a/b/(e)qual b
a: File taxes.
b: Mow lawn.
More important? a/b/(e)qual a
New top priority: TODO(Edit footage.)


In [60]:
my_heap.insert_todo("String guitar.")

a: Edit footage.
b: String guitar.
More important? a/b/(e)qual a


In [44]:
my_heap.show_top()

TODO(File taxes.)


In [45]:
my_heap.insert_todo("Z")

a: File taxes.
b: Z
More important? a/b/(e)qual a


In [46]:
my_heap.show_top()

TODO(File taxes.)


In [47]:
my_heap.insert_todo("Wash car.")

a: File taxes.
b: Wash car.
More important? a/b/(e)qual a


In [48]:
my_heap.show_top()

TODO(File taxes.)
